In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Load the Excel file, skipping the first row (header row with labels)
file_path = r"C:\Users\SMate\Desktop\NASA_SPACEAPPS\space_apps_2024_seismic_detection\space_apps_2024_seismic_detection\data\lunar\test\data\S16_GradeA\xa.s16.00.mhz.1972-09-10HR00_evid00075.csv"
df = pd.read_excel(file_path, skiprows=1, header=None, names=["AbsoluteTime", "RelativeTime", "Velocity"])

# Drop the AbsoluteTime column, keep only relative time and velocity
df = df[["RelativeTime", "Velocity"]]

# Convert to numpy array for easier processing
data = df.to_numpy()

# Extract relative times and velocities
relative_times = data[:, 0]  # Relative time in seconds (starts from 0.0, increases with time)
velocities = data[:, 1]  # Quake velocity

# Calculate the actual time differences (deltaT) between consecutive measurements
# Starting from the second row
deltaTs = np.diff(relative_times)

# Kalman filter parameters
Q = 0.001  # Process noise covariance (can be tuned)
R = 0.1    # Measurement noise covariance (can be tuned)

# Initialize Kalman filter arrays
n_samples = len(velocities)
x = np.zeros(n_samples)  # The estimated true signal (velocity)
P = np.zeros(n_samples)  # State covariance estimate

# Initialize the Kalman filter with the first observed value
x[0] = velocities[0]  # Initial estimate for velocity
P[0] = 1.0  # Initial state covariance

# Apply Kalman filter using actual time differences (deltaT)
for t in range(1, n_samples):
    # Time difference between current and previous measurements
    deltaT = deltaTs[t - 1] if t < len(deltaTs) else deltaTs[-1]  # Handle edge case at the end
    
    # Predict step (state transition model considering deltaT)
    x_pred = x[t-1]  # For simplicity, we assume the velocity remains constant (change model if needed)
    P_pred = P[t-1] + Q * deltaT  # Covariance update scaled by deltaT

    # Kalman gain
    K = P_pred / (P_pred + R)

    # Update step (incorporate new measurement)
    x[t] = x_pred + K * (velocities[t] - x_pred)
    P[t] = (1 - K) * P_pred

# Plotting the results
plt.figure(figsize=(10, 6))
plt.plot(relative_times, velocities, label="Noisy Seismic Data", alpha=0.6)
plt.plot(relative_times, x, label="Kalman Filter Estimate", linewidth=2)
plt.legend()
plt.title("Kalman Filter Applied to Seismic Data with Varying Time Steps")
plt.xlabel("Relative Time (seconds)")
plt.ylabel("Quake Velocity")
plt.show()


ValueError: Excel file format cannot be determined, you must specify an engine manually.